# Notebook 9: Gradio Web Interface
## User interface for Job Application Agent

**Purpose**: Create web interface for deployment

**Dependencies**: Notebooks 1-7


## Installation

In [ ]:
!pip install gradio torch numpy pandas -q
import gradio as gr
import pickle
import json
from datetime import datetime

print("✓ Gradio setup complete")

## Load Modules

In [ ]:
# Load agent and related classes
try:
    with open('/tmp/application_agent_module.pkl', 'rb') as f:
        agent_data = pickle.load(f)
    JobApplicationAgent = agent_data['JobApplicationAgent']
    print("✓ Agent loaded")
except Exception as e:
    print(f"⚠️ Agent not found: {e}")

try:
    with open('/tmp/job_scraper_module.pkl', 'rb') as f:
        scraper_data = pickle.load(f)
    sample_jobs = scraper_data['sample_jobs']
    print("✓ Sample jobs loaded")
except:
    sample_jobs = []

## Gradio Interface Functions

In [ ]:
# Global agent instance
agent = None

def initialize_agent(resume_text):
    """Initialize agent with resume"""
    global agent
    try:
        agent = JobApplicationAgent(resume_text=resume_text)
        return "✓ Agent initialized with resume"
    except Exception as e:
        return f"Error: {str(e)}"

def search_and_analyze_jobs(keywords, location, num_results):
    """
    Search for jobs and show analysis.
    """
    try:
        if not agent:
            return "❌ Please initialize agent with resume first", ""
        
        num_results = int(num_results)
        
        # Run workflow
        results = agent.run_workflow(
            keywords=keywords,
            location=location,
            num_results=min(num_results, 50),
            threshold=0.6,
            max_applications=10
        )
        
        # Format output
        output = f"""## Job Search Results

**Search Parameters:**
- Keywords: {keywords}
- Location: {location}
- Results: {num_results}

**Statistics:**
- Total Jobs Found: {results['summary']['jobs_found']}
- Relevant Jobs: {results['summary']['jobs_relevant']}
- Applications Prepared: {results['summary']['applications_prepared']}
- Pass Rate: {results['summary']['pass_rate']:.1%}
- Avg Relevance Score: {results['summary']['avg_relevance_score']:.3f}

### Top Opportunities:
"""
        
        for i, app in enumerate(results['applications'][:5], 1):
            job = app['job']
            score = app['relevance_score']
            output += f"""
{i}. **{job['title']}** at {job['company']}
   - Location: {job['location']}
   - Relevance Score: {score:.1%}
   - Salary: {job.get('salary', 'Not specified')}
   - Skills: {', '.join(job.get('skills', [])[:3])}
"""
        
        return output, json.dumps(results, indent=2, default=str)
    
    except Exception as e:
        return f"❌ Error: {str(e)}", ""

def customize_resume(job_title, job_company):
    """
    Show customized resume for a job.
    """
    try:
        if not agent or not agent.applications:
            return "❌ Please search for jobs first"
        
        # Find matching application
        for app in agent.applications:
            if app['job']['title'] == job_title and app['job']['company'] == job_company:
                output = f"""## Customized Resume

**For Position:** {job_title} at {job_company}

**Relevance Score:** {app['relevance_score']:.1%}

### Professional Summary:
{app['customized_resume'].get('summary', 'N/A')}

### Customized Experience:
"""
                for exp in app['customized_resume'].get('experience', []):
                    output += f"""
**{exp.get('role')}** at {exp.get('company')}
- {', '.join(exp.get('achievements', []))}
"""
                
                output += """\n### Prioritized Skills:
"""
                skills = app['customized_resume'].get('skills', [])
                output += f"- {', '.join(skills)}\n"
                
                output += f"""\n### Cover Letter:\n
{app['cover_letter']}"""
                
                return output
        
        return "❌ Job not found in applications"
    
    except Exception as e:
        return f"❌ Error: {str(e)}"

def get_job_details(keywords, location):
    """
    Show detailed job analysis.
    """
    return f"""## Detailed Analysis

### Search Query:
- Keywords: {keywords}
- Location: {location}

### Next Steps:
1. Go to 'Job Search' tab
2. Click 'Search & Analyze'
3. Review recommendations
4. Go to 'Customize Resume' tab
5. Select a job to customize
6. Export or copy customized materials
"""

print("✓ Interface functions created")

## Create Gradio Interface

In [ ]:
with gr.Blocks(title="🚀 AI Job Application Agent", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🚀 AI Job Application Agent
    
    **Powered by MAYINI Framework & Machine Learning**
    
    Automate your job search and resume customization using AI!
    """)
    
    with gr.Tabs():
        
        # Tab 1: Setup
        with gr.TabItem("📋 Setup Resume"):
            gr.Markdown("### Initialize the Agent")
            
            resume_input = gr.Textbox(
                label="Your Resume",
                placeholder="Paste your resume text here...",
                lines=10,
                info="Paste your complete resume text for customization"
            )
            
            init_btn = gr.Button("🎯 Initialize Agent", variant="primary", size="lg")
            init_status = gr.Textbox(label="Status", interactive=False)
            
            init_btn.click(
                fn=initialize_agent,
                inputs=[resume_input],
                outputs=[init_status]
            )
        
        # Tab 2: Job Search
        with gr.TabItem("🔍 Job Search"):
            gr.Markdown("### Search & Analyze Jobs")
            
            with gr.Row():
                with gr.Column():
                    keywords_input = gr.Textbox(
                        label="Job Keywords",
                        placeholder="e.g., Python Developer, Data Scientist",
                        lines=1
                    )
                    location_input = gr.Textbox(
                        label="Location",
                        placeholder="e.g., San Francisco, Remote",
                        lines=1
                    )
                    num_results = gr.Slider(
                        minimum=5,
                        maximum=100,
                        value=20,
                        step=5,
                        label="Number of Results"
                    )
            
            search_btn = gr.Button("🔍 Search & Analyze", variant="primary", size="lg")
            
            with gr.Column():
                jobs_output = gr.Markdown(label="Results")
                json_output = gr.Textbox(
                    label="Raw JSON",
                    interactive=False,
                    lines=5
                )
            
            search_btn.click(
                fn=search_and_analyze_jobs,
                inputs=[keywords_input, location_input, num_results],
                outputs=[jobs_output, json_output]
            )
        
        # Tab 3: Resume Customization
        with gr.TabItem("📝 Customize Resume"):
            gr.Markdown("### Customize Resume for Job")
            
            with gr.Row():
                job_title_input = gr.Textbox(
                    label="Job Title",
                    placeholder="Select a job title from search results",
                    lines=1
                )
                job_company_input = gr.Textbox(
                    label="Company",
                    placeholder="Company name",
                    lines=1
                )
            
            customize_btn = gr.Button("✨ Customize Resume", variant="primary", size="lg")
            
            customized_output = gr.Markdown(label="Customized Resume")
            
            customize_btn.click(
                fn=customize_resume,
                inputs=[job_title_input, job_company_input],
                outputs=[customized_output]
            )
        
        # Tab 4: About
        with gr.TabItem("ℹ️ About"):
            gr.Markdown("""
            ## About This Application
            
            This AI-powered job application agent helps you:
            
            1. **Search** for relevant job postings
            2. **Analyze** job descriptions using machine learning
            3. **Customize** your resume for each position
            4. **Generate** tailored cover letters
            5. **Track** your applications
            
            ### Technologies Used:
            - **MAYINI Framework**: Custom ML models for job classification
            - **Neural Networks**: 300-dim job embeddings + classifier
            - **Gradio**: Web interface
            - **Hugging Face**: Deployment platform
            
            ### Features:
            - ✅ Privacy-first: All processing local
            - ✅ Fast: Optimized inference (ML model)
            - ✅ Free: Open-source
            - ✅ Smart: ML-powered job filtering
            - ✅ Scalable: Can handle 100+ jobs
            
            ### How to Use:
            1. Go to "Setup Resume" tab
            2. Paste your resume and initialize
            3. Go to "Job Search" tab
            4. Enter keywords and location
            5. Review results with relevance scores
            6. Go to "Customize Resume" tab
            7. Select a job to customize
            8. Download or copy the customized materials
            """)

print("✓ Gradio interface created")

## Testing Interface (Local)

In [ ]:
print("\n" + "="*60)
print("GRADIO INTERFACE READY")
print("="*60)
print("\nTo launch the interface:")
print("  demo.launch(share=False)")
print("\nThen visit: http://localhost:7860")
print("\nFor Hugging Face Spaces:")
print("  - Save this as app.py")
print("  - Push to HF Space repository")
print("  - Interface will auto-deploy")
print("\n" + "="*60)

## Export Interface

In [ ]:
# Save interface definition
interface_data = {
    'demo': demo,
    'functions': {
        'initialize_agent': initialize_agent,
        'search_and_analyze_jobs': search_and_analyze_jobs,
        'customize_resume': customize_resume,
        'get_job_details': get_job_details,
    }
}

with open('/tmp/gradio_interface.pkl', 'wb') as f:
    pickle.dump(interface_data, f)

print("✓ Gradio interface exported to /tmp/gradio_interface.pkl")

# Also generate app.py code for deployment
app_code = '''import gradio as gr
import pickle
import json
from datetime import datetime

# Load modules (would be in production)
# from src.application_agent import JobApplicationAgent

print("✓ Gradio App Ready for Deployment")
print("✓ This code runs on Hugging Face Spaces")
print("✓ All inference is done locally")

# Create app function
def create_app():
    # Interface creation code here
    with gr.Blocks(title="🚀 AI Job Application Agent") as demo:
        gr.Markdown("""
        # 🚀 AI Job Application Agent
        Powered by MAYINI Framework
        """)
        # ... rest of interface
    return demo

# Create demo
demo = create_app()

if __name__ == "__main__":
    demo.launch(
        share=False,
        server_name="0.0.0.0",
        server_port=7860
    )
'''

with open('/tmp/app_example.py', 'w') as f:
    f.write(app_code)

print("✓ Example app.py saved to /tmp/app_example.py")

## Summary

✅ **Notebook 9 Complete**

### Features:
- Resume setup and initialization
- Job search interface
- Resume customization
- Documentation tab
- Gradio integration
- HF Spaces ready

**Ready for complete demo (Notebook 10)**